In [1]:
def get_eeg_svm_coefs(X, Y, multiclass = 'binary'):
    ''' Running SVM  
    :param X: if 2D data (trials X time) if 3D (trials X channels X time)
    :param Y: targets (length should be as 0th dimension of X)
    :return coefs: coefficient arrays for all available classes
    :return duration: time that it took to run the procedure
    
    >>> coefs,duration=get_eeg_svm_coefs(X,Y)
    '''
    
    import time
    from sklearn.preprocessing import scale
    from sklearn.svm import LinearSVC
    import itertools
    from scipy.spatial.distance import pdist, squareform
    import numpy as np
    from IPython.core.debugger import set_trace
    
    t=time.time()
    dims = X.shape
#     set_trace()
    if len(dims) == 3:
        dataIn = scale(np.reshape(X,(X.shape[0],X.shape[1]*X.shape[2]),'C'))
#         set_trace()
    else:
        dataIn = scale(X)
    labelsIn = Y
    
    if multiclass == 'binary':       
        coefs = list()
        coefs_scaled = list() # coefs scaled by covariance for weight projection
        clf = LinearSVC(C=1.0)
        nums = len(list(itertools.combinations(np.unique(labelsIn), 2)))
    #     set_trace()
        for idx, i in enumerate(itertools.combinations(np.unique(labelsIn), 2)):
    #         print(i)
            X = dataIn[np.logical_or(labelsIn==i[0], labelsIn==i[1]),:]
            Y = labelsIn[np.logical_or(labelsIn==i[0], labelsIn==i[1])]
            clf.fit(X,Y)
    #         set_trace()
            if len(dims) == 3:
                coeftemp  = np.reshape(clf.coef_,(dims[1],dims[2]),'C')
                coeftemp2 = np.dot(np.cov(X.transpose()),clf.coef_.transpose())
                coeftemp2 = np.reshape(coeftemp2.transpose(),(dims[1],dims[2]),'C')
            else:
                coeftemp  = clf.coef_
                coeftemp2 = np.dot(np.cov(X.transpose()),clf.coef_.transpose()).transpose()

    #         set_trace()
            coefs.append(coeftemp)
            coefs_scaled.append(coeftemp2)
    elif multiclass == 'multi':
        coefs = list()
        coefs_scaled = list() # coefs scaled by covariance for weight projection
        clf = LinearSVC(C=1.0, multi_class = 'ovr')
        clf.fit(dataIn,labelsIn)
        if len(dims) == 3:
            coeftemp  = np.reshape(clf.coef_,(dims[1],dims[2]),'C')
            coeftemp2 = np.dot(np.cov(X.transpose()),clf.coef_.transpose())
            coeftemp2 = np.reshape(coeftemp2.transpose(),(dims[1],dims[2]),'C')
        else:
            coeftemp  = clf.coef_
            coeftemp2 = np.dot(np.cov(X.transpose()),clf.coef_.transpose()).transpose()

#         set_trace()
        coefs.append(coeftemp)
        coefs_scaled.append(coeftemp2)


    duration=time.time() - t
    duration=duration/60
    return coefs, coefs_scaled, duration

In [2]:
def get_eeg_svm_input(eegData, behavData, condition, times, timebin, jitCorrectDataFile = None):
    ''' get imput for SVM  
    :param eegData:   epochs from the -epo.fif 
    :param behavData: data frame with behavioral data
    :param condition: string 'Face','Color','Value','FaceValue','ColorValue','Response'
    :times:           tuple defining time window in seconds (timeStart, timeEnd)
    :timebin:         number of time points to average over
    
    :return X:        3D array input for classification (observations X channels X time)
    :return y:        labels vector to the length of first dimension of X
    :return time:     time index
    >>> X,y, time =get_eeg_svm_input(eegData, behavData, times, timebin, n_avg)
    '''
    import mne
    import numpy as np
    
    sfreq = eegData.info['sfreq']
    if jitCorrectDataFile == None:
        AllEpochs = eegData.copy().crop(times[0],times[1]).get_data()
    else:
        del eegData
        temp = np.load(jitCorrectDataFile)
        timeVect = np.linspace(-.2,2.2,temp.shape[2])
        t0 = find_nearest_idx(timeVect,times[0])
        tE = find_nearest_idx(timeVect,times[1])
        AllEpochs = temp[:,:,t0:tE+1]
        del temp, timeVect
#     AllEvents = eegData.events
#     set_trace()
    if condition == 'Face' or condition == 'Color':
        labels = [1,2,3,4,5]
        n_labels = len(labels)
        n_avg = 4
        for f in range(n_labels):
#             set_trace()
            temp = AllEpochs[behavData.Trial[behavData[condition] == f+1]-1,:,:] #-1 to start trial index from 0
#             temp = AllEpochs[AllEvents[:,2]==labels[f]+101,:,:] #
            #average trials
            nChunk  = int(temp.shape[0]/n_avg)

            if timebin == 0:
                condAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
                for c in range(nChunk):
                    condAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)

            else:
                nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
                n_tpnt = int(timebin/(1000/sfreq))
                condAvg= np.zeros((nChunk,temp.shape[1],nBin))
                for c in range(nChunk):
                    temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    for t in range(nBin):
                        condAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)           
            if f>0:
                X = np.concatenate((X,condAvg), axis = 0)
            else:
                X = condAvg

        y = np.concatenate((np.ones((1,nChunk)),2*np.ones((1,nChunk)),3*np.ones((1,nChunk)),
                                    4*np.ones((1,nChunk)),5*np.ones((1,nChunk))),axis = 1)                 
        # X = np.reshape(X, (X.shape[0], X.shape[1]*X.shape[2]), order='F')
        y = np.int32(y[0])

    elif condition == 'Value':
        newValues = [0]*5
        newValues[0] = [-100,-70,-50]
        newValues[1] = [-40,-30,-20]
        newValues[2] = [0,0,0]
        newValues[3] = [20,30,40]
        newValues[4] = [50,70,100]
        y = np.empty((0))
        n_avg = 3
        for v in range(len(newValues)):
            temp = AllEpochs[(behavData.Value == newValues[v][0])|
                             (behavData.Value == newValues[v][1])|
                             (behavData.Value == newValues[v][2]) ,:,:] #-1 to start trial index from 0
            # print(temp.shape[0])
            #average trials    
            nChunk  = int(temp.shape[0]/n_avg)
            if timebin == 0:
                valueAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
                for c in range(nChunk):
                    valueAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    y = np.append(y,v)
            else:
                nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
                n_tpnt = int(timebin/(1000/sfreq))
                valueAvg= np.zeros((nChunk,temp.shape[1],nBin))
                for c in range(nChunk):
                    temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    for t in range(nBin):
                        valueAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)    
                    y = np.append(y,v)
            if v>0:
                X = np.concatenate((X,valueAvg), axis = 0)
            else:
                X = valueAvg
    elif condition == 'Value-2':
        newValues = [0]*5
        newValues[0] = [-100,-70] # [-100,-70,-50]
        newValues[1] = [-30,-20]# [-40,-30,-20]
        newValues[2] = [0,0,0]
        newValues[3] = [20,30]#[20,30,40]
        newValues[4] = [70,100] ##[50,70,100]        y = np.empty((0))
        
        y = np.empty((0))
        n_avg = 3
        for v in range(len(newValues)):
            temp = AllEpochs[(behavData.Value == newValues[v][0])|
                             (behavData.Value == newValues[v][1]),:,:] #|(behavData.Value == newValues[v][2])
            # print(temp.shape[0])
            #average trials    
            nChunk  = int(temp.shape[0]/n_avg)
            if timebin == 0:
                valueAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
                for c in range(nChunk):
                    valueAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    y = np.append(y,v)
            else:
                nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
                n_tpnt = int(timebin/(1000/sfreq))
                valueAvg= np.zeros((nChunk,temp.shape[1],nBin))
                for c in range(nChunk):
                    temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    for t in range(nBin):
                        valueAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)    
                    y = np.append(y,v)
            if v>0:
                X = np.concatenate((X,valueAvg), axis = 0)
            else:
                X = valueAvg

    elif condition == 'FaceValue' or condition == 'ColorValue':
        values = np.unique(behavData[condition])
        y = np.empty((0))
        n_avg = 4

        for v in range(len(values)):
            temp = AllEpochs[behavData.Trial[behavData[condition] == values[v]]-1,:,:] #-1 to start trial index from 0
            # print(temp.shape[0])
            #average trials
            nChunk  = int(temp.shape[0]/n_avg)
            if timebin == 0:
                valueAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
                for c in range(nChunk):
                    valueAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    y = np.append(y,values[v])
            else:
                nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
                n_tpnt = int(timebin/(1000/sfreq))
                valueAvg= np.zeros((nChunk,temp.shape[1],nBin))
                for c in range(nChunk):
                    temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    for t in range(nBin):
                        valueAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)    
                    y = np.append(y,values[v])
            if v>0:
                X = np.concatenate((X,valueAvg), axis = 0)
            else:
                X = valueAvg
    
    elif condition == 'Response':
        resps = [0,1]
        y = np.empty((0))
        for r in range(len(resps)):
            temp = AllEpochs[behavData.Trial[behavData.Resp == resps[r]]-1,:,:] #-1 to start trial index from 0
#             print(temp.shape[0])
            #average trials
            n_avg = 4
            nChunk  = int(temp.shape[0]/n_avg)
            if timebin == 0:
                respAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
                for c in range(nChunk):
                    respAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    Y = np.append(Y,resps[r])
            else:
                nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
                n_tpnt = int(timebin/(1000/sfreq))
                respAvg= np.zeros((nChunk,temp.shape[1],nBin))
                for c in range(nChunk):
                    temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    for t in range(nBin):
                        respAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)    
                    y = np.append(y,resps[r])
            if r>0:
                X = np.concatenate((X,respAvg), axis = 0)
            else:
                X = respAvg
    else:
        print('Condition variable not found')
        X = 0
        y = 0
    
    #get time vector
#     set_trace()
    time = np.linspace(times[0],times[1],X.shape[-1])

    return X, y, time

In [3]:
def get_eeg_svm_input_VaueBinary(eegData, behavData, condition, times, timebin, jitCorrectDataFile = None):
    ''' get imput for SVM  
    :param eegData:   epochs from the -epo.fif 
    :param behavData: data frame with behavioral data
    :param condition: string 'Face','Color','Value','FaceValue','ColorValue','Response'
    :times:           tuple defining time window in seconds (timeStart, timeEnd)
    :timebin:         number of time points to average over
    
    :return X:        3D array input for classification (observations X channels X time)
    :return y:        labels vector to the length of first dimension of X
    :return time:     time index
    >>> X,y, time =get_eeg_svm_input(eegData, behavData, times, timebin, n_avg)
    '''
    import mne
    import numpy as np
    
    sfreq = eegData.info['sfreq']
    if jitCorrectDataFile == None:
        AllEpochs = eegData.copy().crop(times[0],times[1]).get_data()
    else:
        del eegData
        temp = np.load(jitCorrectDataFile)
        timeVect = np.linspace(-.2,2.2,temp.shape[2])
        t0 = find_nearest_idx(timeVect,times[0])
        tE = find_nearest_idx(timeVect,times[1])
        AllEpochs = temp[:,:,t0:tE+1]
        del temp, timeVect
#     AllEvents = eegData.events
#     set_trace()
    if condition == 'binValue':
        newValues = [0]*2
        newValues[0] = [-100,-70,-50]
#         newValues[1] = [-40,-30,-20]
#         newValues[2] = [0,0,0]
#         newValues[3] = [20,30,40]
        newValues[1] = [50,70,100]
        y = np.empty((0))
        n_avg = 4
        for v in range(len(newValues)):
            temp = AllEpochs[(behavData.Value == newValues[v][0])|
                             (behavData.Value == newValues[v][1])|
                             (behavData.Value == newValues[v][2]) ,:,:] #-1 to start trial index from 0
            # print(temp.shape[0])
            #average trials    
            nChunk  = int(temp.shape[0]/n_avg)
            if timebin == 0:
                valueAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
                for c in range(nChunk):
                    valueAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    y = np.append(y,v)
            else:
                nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
                n_tpnt = int(timebin/(1000/sfreq))
                valueAvg= np.zeros((nChunk,temp.shape[1],nBin))
                for c in range(nChunk):
                    temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    for t in range(nBin):
                        valueAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)    
                    y = np.append(y,v)
            if v>0:
                X = np.concatenate((X,valueAvg), axis = 0)
            else:
                X = valueAvg

    elif condition == 'binFaceValue' or condition == 'binColorValue':
#         values = [np.unique(behavData[condition])]
#         values = [0,20,50]
        y = np.empty((0))
        n_avg = 4
        values = [0]*2
        values[0] = [20,50]
        values[1] = [-20,-50]

        for v in range(len(values)):
            temp = AllEpochs[(behavData[condition[3:]] == values[v][0])|
                             (behavData[condition[3:]] == values[v][1]),:,:] #-1 to start trial index from 0
            # print(temp.shape[0])
            #average trials
            nChunk  = int(temp.shape[0]/n_avg)
            if timebin == 0:
                valueAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
                for c in range(nChunk):
                    valueAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    y = np.append(y,values[v])
            else:
                nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
                n_tpnt = int(timebin/(1000/sfreq))
                valueAvg= np.zeros((nChunk,temp.shape[1],nBin))
                for c in range(nChunk):
                    temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
                    for t in range(nBin):
                        valueAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)    
                    y = np.append(y,v)
            if v>0:
                X = np.concatenate((X,valueAvg), axis = 0)
            else:
                X = valueAvg
    
#     elif condition == 'Response':
#         resps = [0,1]
#         y = np.empty((0))
#         for r in range(len(resps)):
#             temp = AllEpochs[behavData.Trial[behavData.Resp == resps[r]]-1,:,:] #-1 to start trial index from 0
# #             print(temp.shape[0])
#             #average trials
#             n_avg = 4
#             nChunk  = int(temp.shape[0]/n_avg)
#             if timebin == 0:
#                 respAvg = np.zeros((nChunk,temp.shape[1],temp.shape[2]))
#                 for c in range(nChunk):
#                     respAvg[c,:,:] = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
#                     Y = np.append(Y,resps[r])
#             else:
#                 nBin   = int((times[1]-times[0])*1000/timebin) # number of bins
#                 n_tpnt = int(timebin/(1000/sfreq))
#                 respAvg= np.zeros((nChunk,temp.shape[1],nBin))
#                 for c in range(nChunk):
#                     temp1 = np.mean(temp[c*n_avg:(c+1)*n_avg,:,:],0)
#                     for t in range(nBin):
#                         respAvg[c,:,t] = np.mean(temp1[:,t*n_tpnt:(t+1)*n_tpnt],1)    
#                     y = np.append(y,resps[r])
#             if r>0:
#                 X = np.concatenate((X,respAvg), axis = 0)
#             else:
#                 X = respAvg
    else:
        print('Condition variable not found')
        X = 0
        y = 0
    
    #get time vector
    time = np.linspace(times[0],times[1],X.shape[-1])

    return X, y, time

In [4]:
def get_eeg_behav_data(subj):
    
    import mne
    import os.path as op
    import os
    import pandas as pd
    
    #get study info and paths
    study        = 'MADeEEG1'
    studyPathInd = os.getcwd().find(study)
    dataPath     = op.join('Results',subj)
    behavPath = op.join(os.getcwd()[:studyPathInd+len(study)],'Analysis','Matlab','Behavior','csvfiles')

    #read in processed eeg data
#     file_name= 'AllDataICA-epo.fif'
    file_name= 'stimData-epo.fif'
#     file_name= 'stimDataBaseLine-epo.fif'

    fname = op.join(dataPath,file_name)
    eegData = mne.read_epochs(fname, verbose = 'WARNING')

    #read and organize channels
    chans = eegData.info['ch_names']
    pfrontal = ['Fp1','Fpz','Fp2','AF7','AF3','AFz','AF4','AF8']
    cfrontal = ['FT7','F7','F5','F3','F1','Fz','F2','F4','F6','F8','FT8','FC5','FC3','FC1','FCz','FC2','FC4','FC6']
#     ftemporal= ['F7','FT7','F8','FT8']
    cparietal= ['C5','C3','C1','Cz','C2','C4','C6','CP5','CP3','CP1','CPz','CP2','CP4','CP6']
    ptemporal= ['T7','TP7','T8','TP8']
    oparietal= ['P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10','PO7','PO3','POz','PO4','PO8']
#     otemporal= ['P7','P9','P8','P10']
    occipital= ['O1','Oz','Iz','O2']
    chanbyregion = dict({'pfrontal':pfrontal,'cfrontal':cfrontal,'cparietal':cparietal,'oparietal':oparietal,
                        'occipital':occipital,'ptemporal':ptemporal}) #'ftemporal':[],,'otemporal':[]}
    chanIndex    = dict({'pfrontal':[],'cfrontal':[],'cparietal':[],'oparietal':[],
                        'occipital':[],'ptemporal':[]}) #'ftemporal':[],,'otemporal':[]}
    for r in chanbyregion.keys():
        for c in chanbyregion[r]:
            chanIndex[r].append(chans.index(c))
    chanOrder = []
    for r in chanIndex.keys():
        chanOrder = chanOrder + chanIndex[r]
    channels = dict()
    channels['originalChans'] = chans
    channels['regionalChans'] = chanbyregion
    channels['regionalOrder'] = chanOrder
    
    #read in behavioral data
    behavData = pd.read_csv(op.join(behavPath, subj+'.csv'))
    return eegData, behavData, channels

In [5]:
def get_accuracy_by_time(subj,condition, event, timebin, multiclass='binary'):
    
    import os
    import numpy as np
    import pandas as pd

    #initialize study and path
    study        = 'MADeEEG1'
    studyPathInd = os.getcwd().find(study)
#     def viewResults(subj,condition,event,timebin,plot):
    dataPath     = os.path.join('Results',subj,'classification')#,,'Cluster'
    #load in the preprocessed data
    if multiclass == 'binary':
        pre = 'stim'
    elif multiclass == 'multi':
        pre = 'stimMulti'
    if condition == 'Response' or condition == 'Response-0':
        condWord = condition+'_'
    else:
        condWord = condition+'s_'
            
    if timebin != 0:
        file_name     = pre+'Classification_'+condWord+event+'_'+str(timebin)+'ms.csv'
        permfile_name = pre+'PermPVals_'+condWord+event+'_'+str(timebin)+'ms.csv'
    else:
        file_name     = pre+'Classification_'+condWord+event+'.csv'       
        permfile_name = pre+'PermPVals_'+condWord+event+'.csv'       
    fname    = os.path.join(dataPath,file_name)
    pfname   = os.path.join(dataPath,permfile_name)
#     set_trace()
    acc      = pd.read_csv(fname)
    pval     = pd.read_csv(pfname)
    #extract time
    ttemp = list(acc.columns)
    ttemp = ttemp[1:]
    time  = [float(t) for t in ttemp]
    return time, acc, pval

In [6]:
def get_accuracy_by_time_guggenmos(subj,condition, event, timebin, multiclass='binary'):
    
    import os
    import numpy as np
    import pandas as pd

    #initialize study and path
    study        = 'MADeEEG1'
    studyPathInd = os.getcwd().find(study)
#     def viewResults(subj,condition,event,timebin,plot):
    dataPath     = os.path.join('Results',subj,'classification_guggenmos')#,,'Cluster'
    #load in the preprocessed data
    if multiclass == 'binary':
        pre = 'stim'
    elif multiclass == 'multi':
        pre = 'stimMulti'
    if condition == 'Response' or condition == 'Response-0' or condition == 'Value-2':
        condWord = condition+'_'
    else:
        condWord = condition+'s_'
            
    if timebin != 0:
        file_name     = pre+'Classification_'+condWord+event+'_'+str(timebin)+'ms.npy'
        permfile_name = pre+'PermPVals_'+condWord+event+'_'+str(timebin)+'ms.npy'
    else:
        file_name     = pre+'Classification_'+condWord+event+'.npy'       
        permfile_name = pre+'PermPVals_'+condWord+event+'.npy'       
    fname    = os.path.join(dataPath,file_name)
    pfname   = os.path.join(dataPath,permfile_name)
#     set_trace()
    acc      = np.load(fname)
#     pval     = pd.read_csv(pfname)
    #extract time
#     ttemp = list(acc.columns)
#     ttemp = ttemp[1:]
#     time  = [float(t) for t in ttemp]
    return acc

In [7]:
def get_dissimilarity_by_time_guggenmos(subj,condition, event, timebin, measure,multiclass='binary'):
    
    import os
    import numpy as np
    import pandas as pd

    #initialize study and path
    study        = 'MADeEEG1'
    studyPathInd = os.getcwd().find(study)
#     def viewResults(subj,condition,event,timebin,plot):
    dataPath     = os.path.join('Results',subj,'classification_guggenmos')#,,'Cluster'
    #load in the preprocessed data
#     if multiclass == 'binary':
#         pre = 'stim'
#     elif multiclass == 'multi':
#         pre = 'stimMulti'
    if condition == 'Response' or condition == 'Response-0' or condition == 'Value-2':
        condWord = condition+'_'
    else:
        condWord = condition+'s_'
            
#     if timebin != 0:
    if measure.lower() == 'dvWeighted':
        file_name     = 'rdm_'+condWord+event+'_'+str(timebin)+'ms.npy'
    elif measure.lower() == 'euclidean':
        file_name     = 'eucDistance_'+condWord+event+'_'+str(timebin)+'ms.npy'
    elif measure.lower() == 'pearson':
        file_name     = 'pscDistance_'+condWord+event+'_'+str(timebin)+'ms.npy'

#         permfile_name = pre+'PermPVals_'+condWord+event+'_'+str(timebin)+'ms.npy'
#     else:
#         file_name     = 'rdm_'+condWord+event+'.npy'       
#         permfile_name = pre+'PermPVals_'+condWord+event+'.npy'       
    fname    = os.path.join(dataPath,file_name)
#     pfname   = os.path.join(dataPath,permfile_name)
#     set_trace()
    dissim      = np.load(fname)
#     pval     = pd.read_csv(pfname)
    #extract time
#     ttemp = list(acc.columns)
#     ttemp = ttemp[1:]
#     time  = [float(t) for t in ttemp]
    return  dissim

In [8]:
def find_nearest_idx(array, value):
    import numpy as np
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


In [9]:
def plot_accuracy_by_time(subj, condition, timewin, timebin, overlay = 0, axes = None, multiclass = 'binary'):
    
    import sys
    
    if timewin[0] >= .8:
        event = 'Dot'
    else:
        event = 'Stimulus'
    if overlay > 3:
        sys.exit('Error!')
    #possible line colors
    clrs = ['royalblue','salmon','skyblue','g']        
    time, accuracy, pval = get_accuracy_by_time(subj,condition, event, timebin,multiclass = multiclass)
    
    #make significance horizental line
    if multiclass == 'binary':
        chance = 50
    elif multiclass == 'multi':
        chance = 20
    
    sigLine = np.empty(pval.iloc[0,1:].shape)
    sigLine[:] = np.nan
    sigLine[pval.iloc[0,1:]<=.05] = chance
#     set_trace()
    # plot accuracy
    if axes == None:
        plt.plot(time, np.mean(np.asarray(accuracy.iloc[:,1:]),axis = 0), color = clrs[overlay])
        plt.plot(time, sigLine, color = clrs[overlay], alpha = .5)
        axes = plt.axes()
    else:
        axes.plot(time, np.mean(np.asarray(accuracy.iloc[:,1:]),axis = 0),color = clrs[overlay],label = condition)
        axes.plot(time, sigLine, color = clrs[overlay],  marker='o',  markersize=5, alpha = .5)

    plt.xlabel('time ms')
    plt.ylabel('accuracy%')
    plt.title('accuracy by time')
    return axes

In [10]:
def plot_accuracy_by_time_guggenmos(subj, condition, timewin, timebin, overlay = 0, axes = None, multiclass = 'binary'):
    
    import sys
    
    if timewin[0] >= .8:
        event = 'Dot'
        times = (.8,2)
    else:
        event = 'Stimulus'
        times = (-.2,1)
    if overlay > 3:
        sys.exit('Error!')
    #possible line colors
    clrs = ['royalblue','salmon','skyblue','g']   
    accuracy = get_accuracy_by_time_guggenmos(subj,condition, event, timebin,multiclass = multiclass)
    time = np.arange(times[0]*1000,times[1]*1000,timebin)
    #make significance horizental line
    if multiclass == 'binary':
        chance = 50
    elif multiclass == 'multi':
        chance = 20
    
#     sigLine = np.empty(pval.iloc[0,1:].shape)
#     sigLine[:] = np.nan
#     sigLine[pval.iloc[0,1:]<=.05] = chance
#     set_trace()
    # plot accuracy
    if axes == None:
        plt.plot(time, np.nanmean(accuracy,axis = (0,1)), color = clrs[overlay])
#         plt.plot(time, sigLine, color = clrs[overlay], alpha = .5)
        axes = plt.axes()
    else:
        axes.plot(time, np.nanmean(accuracy,axis = (0,1)),color = clrs[overlay],label = condition)
#         axes.plot(time, sigLine, color = clrs[overlay],  marker='o',  markersize=5, alpha = .5)

    plt.xlabel('time ms')
    plt.ylabel('accuracy%')
    plt.title('accuracy by time')
    return axes

In [11]:
def plot_dissimilarity_by_time_guggenmos(subj, condition, timewin, timebin, overlay = 0, axes = None, multiclass = 'binary'):
    
    import sys
    
    if timewin[0] >= .8:
        event = 'Dot'
        times = (.8,2)
    else:
        event = 'Stimulus'
        times = (-.2,1)
    if overlay > 3:
        sys.exit('Error!')
    #possible line colors
    clrs = ['royalblue','salmon','skyblue','g']   
    dissim = get_dissimilarity_by_time_guggenmos(subj,condition, event, timebin,multiclass = multiclass)
    time = np.arange(times[0]*1000,times[1]*1000,timebin)
    #make significance horizental line
#     if multiclass == 'binary':
#         chance = 50
#     elif multiclass == 'multi':
#         chance = 20
    
#     sigLine = np.empty(pval.iloc[0,1:].shape)
#     sigLine[:] = np.nan
#     sigLine[pval.iloc[0,1:]<=.05] = chance
#     set_trace()
    # plot accuracy
    if axes == None:
        plt.plot(time, np.nanmean(dissim,axis = (0,1)), color = clrs[overlay])
#         plt.plot(time, sigLine, color = clrs[overlay], alpha = .5)
        axes = plt.axes()
    else:
        axes.plot(time, np.nanmean(dissim,axis = (0,1)),color = clrs[overlay],label = condition)
#         axes.plot(time, sigLine, color = clrs[overlay],  marker='o',  markersize=5, alpha = .5)

    plt.xlabel('time ms')
    plt.ylabel('dissim')
    plt.title('dissimilarity by time')
    return axes

In [12]:
def plot_coef_heatmap (coefmean, time, channels, cmap = 'jet', vmin= None, vmax = None):
    xticks = np.around(np.linspace(time[0],time[-1],8),decimals = 2)
    timelabels = []
    yticks = range(len(channels['originalChans']))
#     set_trace()
    [timelabels.append(str(t)) for t in xticks]
    chanlabels = []
    [chanlabels.append(channels['originalChans'][ch]) for ch in channels['regionalOrder']]
    chanlabels = reversed(chanlabels)

    plt.imshow(coefmean[channels['regionalOrder'],:],cmap=cmap,
               extent=[xticks.min(), xticks.max(),yticks[0], yticks[-1]],
               aspect='auto',origin='upper',vmin=vmin, vmax=vmax) #,vmin=-.006, vmax=.006
    ax = plt.gca()
    plt.xticks(xticks)
    plt.yticks(yticks)
    ax.set_xticklabels(timelabels,fontdict={'fontsize':8})
    ax.set_xlabel('time (ms)')
    ax.set_yticklabels(chanlabels,fontdict={'fontsize':5,'fontweight': 'bold'})
    ax.set_ylabel('channels')
    plt.title('coefs heatmap')
    plt.colorbar()


In [13]:
def plot_scalpmap(coefmean, infoStructure, time, timeWindow=None, fig=None):
    
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    if timeWindow!= None: 
        ind1 = find_nearest_idx(time, timeWindow[0])
        ind2 = find_nearest_idx(time, timeWindow[1])

        coefPlot  = np.mean(coefMean[:,ind1:ind2],1)
        title_txt = str(timeWindow)+'s scalp map\nvmin='+ str(round(coefPlot.min(),3))+' vmax=' +str(round(coefPlot.max(),3))
    else:
        coefPlot  = coefmean.transpose()
        title_txt = str(round(time*1000,1))+'ms'
    
#     print (eegData.info['nchan'])
#     set_trace()
#     biosemi_layout = mne.channels.read_layout('biosemi')
#     pos = eegData.info
    im, cn = mne.viz.plot_topomap(coefPlot,infoStructure, vmin=np.min(coefPlot) , cmap='jet')
    ax = plt.gca()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="5%", pad="3%",)

    if fig != None:
        fig.colorbar(im,cax = cax, orientation="horizontal")
        cax.xaxis.set_ticks_position("bottom")

#     set_trace()
#     fig.colorbar()
#     plt.title(title_txt)
#     fig.set_title(title_txt,fontdict={'fontsize':6})
    return title_txt
